In [1]:
import cv2
import skimage.io as io
import numpy as np
from skimage.morphology import binary_erosion, binary_dilation, binary_closing,skeletonize, thin
from scipy.signal import convolve2d
from statistics import mode 
from math import sqrt

In [2]:
def showBinaryImg(img):
    cv2.imshow("Images",img*255)
    cv2.waitKey(0) 
    cv2.destroyAllWindows()
def showImg(img):
    cv2.imshow("Images",img)
    cv2.waitKey(0) 
    cv2.destroyAllWindows()

In [3]:
def calcStaffHeightSpace(img):
    count=0
    staffHeight=[]
    staffSpace=[]
    n,m=img.shape
    lines=[]
    for j in range(m):
        start=end=0
        for i in range(n-1):
            if(img[i][j]!=img[i+1][j]):
                if img[i][j]==0:
                    #end of zeros
                    start=i
                    staffSpace.append(count)
                else:
                    #end of ones
                    end=i
                    lines.append([j,start,end,count])#colIndex,start in coloum Index,end in coloum index,thikness
                    staffHeight.append(count)
                count=0
            else:
                count+=1
        
                       
    staffHeight=mode(staffHeight)
    staffSpace=mode(staffSpace)
    return lines,staffSpace,staffHeight


In [4]:
def removeSymbol(img,lines,staffHeight,staffSpace):
    Thres=min(staffHeight+staffSpace,2*staffHeight)
    lines=np.array(lines)
    indices=np.where(lines[:,3]>Thres)
    staffImg=np.copy(img)
    for i in indices[0]:
       # print(i)
        col,start,end,thres=lines[i]
        staffImg[start:end+1,col]=0
    #print(img)
    return staffImg

In [5]:
img= cv2.imread("Binarized.png",0) 
img=img/255
# print(img)
img=1-img
imgLines,staffS,staffH=calcStaffHeightSpace(img)
# print(staffH,staffS)
imgRemovedSymbols=removeSymbol(img,imgLines,staffH,staffS)
cv2.imwrite("staffout.png",imgRemovedSymbols*255)
#showBinaryImg(imgRemovedSymbols)
neg=img-imgRemovedSymbols
print(img)
print(imgRemovedSymbols)
cv2.imwrite("neg.png",img*255)

[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]
[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]


True

In [6]:
def getDist(curComp,compIndex,lines,j,staffHremoved):
    indices=np.where((lines[:,0]==j) & (lines[:,3] >= 0.5*staffHremoved) & (lines[:,3] <= 2*staffHremoved))
    print(indices)
    if(compIndex>=indices[0].size):
        return 0
    print("outtt")
    nextComp=lines[indices[0][compIndex]]
    colIndex1=curComp[0]
    colIndex2=nextComp[0]
    rowIndex1=(curComp[2]+curComp[1])/2
    rowIndex2=(nextComp[2]+nextComp[1])/2
    dist=sqrt((colIndex1-colIndex2)**2+(rowIndex1-rowIndex2)**2)
    print(rowIndex1,rowIndex2)
    return (1/j)*dist

imgLines,staffSremoved,staffHremoved=calcStaffHeightSpace(imgRemovedSymbols)
lines=np.array(imgLines)
indices=np.where(lines[:,0]==5)
k=10
oGlobal=[]
m=imgRemovedSymbols.shape[1]
for i in range(m):
    o=[]
    indices=np.where((lines[:,0]==i) & (lines[:,3] >= 0.5*staffHremoved) & (lines[:,3] <= 2*staffHremoved))
    if indices[0].size > 5:
        for component in indices[0]:
            # if(i==735):
            #     print(i,lines[component])
            orient=0
            for j in range(k):
                if(col+j+1<m):

                    orient+=getDist(lines[component],component,lines,col+j+1,staffHremoved)
            o.append(orient)
        oi=sum(o)/indices[0].size
        oGlobal.append(oi)

#print(oGlobal)


NameError: name 'col' is not defined